In [ ]:
import os
import time
import torch
import matplotlib.pyplot as plt
import utilities as util

In [ ]:
# load surrogate
surrogate = util.Surrogate()
print(surrogate.model_info["model_in_list"])

In [ ]:
n_samples = 10000
x = surrogate.sample_x(n_samples)
y = surrogate.model(x).detach()

print(torch.min(y[:, 0]))
print(torch.min(y[:, 1]))

In [ ]:
x_x, y_x = x[y[:, 0] < 0], y[y[:, 0] < 0]
x_y, y_y = x[y[:, 1] < 0], y[y[:, 1] < 0]

n_x, n_y = x_x.shape[0], x_y.shape[0]

In [ ]:
# plot histogram
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, figsize=(10, 5))
ax[0].hist(y[:, 0], bins=100)
ax[0].set_xlabel("sigma_x")
ax[1].hist(y[:, 1], bins=100)
ax[1].set_xlabel("sigma_y")

ax[0].set_ylabel("Counts")
ax[0].set_title("Ratio of Negative Samples: {:.2f} %".format(100 * n_x / n_samples))
ax[1].set_title("Ratio of Negative Samples: {:.2f} %".format(100 * n_y / n_samples))

for i in range(2):
    ax[i].axvline(x=0, color='C1', linestyle="--")
fig.tight_layout()
# fig.savefig('output_distribution.png', facecolor='w', transparent=False)

In [ ]:
def normalize(x):
    x_lim = surrogate.raw_x_lim
    norm_x = x - x_lim[:, 0]
    norm_fac = (x_lim[:, 1] - x_lim[:, 0])
    norm_x = torch.where(norm_fac > 0, norm_x / norm_fac, 0.5 * torch.ones_like(norm_x))
    return norm_x

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12, 5))
for i in range(x_x.shape[0]):
    ax[0].plot(normalize(x_x[i]))
for i in range(x_y.shape[0]):
    ax[1].plot(normalize(x_y[i]))

for i in range(2):
    ax[i].set_xlabel("Input Dimension")
ax[0].set_ylabel("Normalized Value")
ax[0].set_title("Inputs for Negative sigma_x")
ax[1].set_title("Inputs for Negative sigma_y")
fig.tight_layout()
# fig.savefig('input_values.png', facecolor='w', transparent=False)

In [ ]:
mean = 0.5 * torch.ones((1, surrogate.x_dim))

diff_x = torch.abs(normalize(x_x) - mean)
max_dim_x = torch.max(diff_x, dim=1)[0]
sort_idx_x = torch.argsort(max_dim_x)
x_x_sorted, y_x_sorted = x_x[sort_idx_x], y_x[sort_idx_x]

diff_y = torch.abs(normalize(x_y) - mean)
max_dim_y = torch.max(diff_y, dim=1)[0]
sort_idx_y = torch.argsort(max_dim_y)
x_y_sorted, y_y_sorted = x_y[sort_idx_y], y_y[sort_idx_y]

# x_mean = torch.mean(x, dim=0)
# x_lb = torch.quantile(x, q=0.05, dim=0)
# x_ub = torch.quantile(x, q=0.95, dim=0)

x_mean = torch.mean(surrogate.raw_x_data, dim=0)
x_lb = torch.quantile(surrogate.raw_x_data, q=0.05, dim=0)
x_ub = torch.quantile(surrogate.raw_x_data, q=0.95, dim=0)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12, 5))
for i in range(3):
    ax[0].plot(normalize(x_x_sorted[i]))
for i in range(3):
    ax[1].plot(normalize(x_y_sorted[i]))

for i in range(2):
    ax[i].plot(normalize(x_mean), "gray")
    ax[i].fill_between(torch.arange(x.shape[1]), normalize(x_lb), normalize(x_ub), color="gray", alpha=0.25)
    ax[i].set_xlabel("Input Dimension")
    ax[i].axhline(y=0, color="gray", linestyle="--")
    ax[i].axhline(y=1.0, color="gray", linestyle="--")

ax[0].set_ylabel("Normalized Value")
ax[0].set_title("Inputs for Negative sigma_x")
ax[1].set_title("Inputs for Negative sigma_y")
fig.tight_layout()
# fig.savefig('input_values_center_raw.png', facecolor='w', transparent=False)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12, 5))
for i in range(3):
    ax[0].plot(normalize(x_x_sorted[-(i+1)]))
for i in range(3):
    ax[1].plot(normalize(x_y_sorted[-(i+1)]))

for i in range(2):
    ax[i].plot(normalize(x_mean), "gray")
    ax[i].fill_between(torch.arange(x.shape[1]), normalize(x_lb), normalize(x_ub), color="gray", alpha=0.25)
    ax[i].set_xlabel("Input Dimension")
    ax[i].axhline(y=0, color="gray", linestyle="--")
    ax[i].axhline(y=1.0, color="gray", linestyle="--")
    
ax[0].set_ylabel("Normalized Value")
ax[0].set_title("Inputs for Negative sigma_x")
ax[1].set_title("Inputs for Negative sigma_y")
fig.tight_layout()
# fig.savefig('input_values_edges_raw.png', facecolor='w', transparent=False)

In [ ]:
# use sample optimum as reference point
dim = 0
unit, unit_factor = "(mm)", 1e3
ground_truth = util.NegativeTransverseBeamSize(surrogate.model)
gt = ground_truth(x).detach()
ref_idx = torch.argmax(gt)
ref_x, ref_y = x[ref_idx], y[ref_idx, dim]


# 1D scan reference point
def scan_ref_x(n_samples, dim=0):
    x_lim = surrogate.raw_x_lim
    samples = ref_x.repeat(n_samples, 1)
    samples[:, dim] = torch.linspace(x_lim[dim, 0], x_lim[dim, 1], n_samples)
    return samples

# Optimum as Reference Point

In [ ]:
n_scan = 100
nrows, ncols = 5, 3
figsize = (12, 12 * nrows / ncols)

fig, ax = plt.subplots(nrows=nrows, ncols=ncols, sharey=False, figsize=figsize)
idx = [i for i in range(surrogate.x_dim) if i not in surrogate.fixed_feature_columns]  # remove fixed dimensions


for i in range(nrows * ncols):
    ax_i = ax[i // ncols, i % ncols]
    if i > len(idx) - 1:
        ax_i.axis('off')
    else:
        j = idx[i]

        # map sim values to pv
        parameter_name = surrogate.model_info["model_in_list"][j]
        pv_name = surrogate.pv_info["sim_name_to_pv_name"][parameter_name]
        pv_factor = surrogate.pv_info["sim_to_pv_factor"][parameter_name]
        pv_unit = surrogate.pv_info["pv_unit"][pv_name]
        
        # calculate 1D scan
        x_scan = scan_ref_x(n_scan, dim=j)
        y_scan = surrogate.model(x_scan).detach()[:, dim]

        # plot data
        ax_i.plot(x_scan[:, j] * pv_factor, y_scan * unit_factor, "C0")
        ax_i.axhline(y=0.0, color="gray", linestyle="--")
        ax_i.set_xlabel("{} {}".format(pv_name, pv_unit))

        if i % 3 == 0:
            label = ["sigma_x", "sigma_y"]
            ax_i.set_ylabel("{} {}".format(label[dim], unit))

fig.tight_layout()
#fig.savefig('function_shape_at_optimum.png', facecolor='w', transparent=False)

# Negative Samples as Reference Point

In [ ]:
# correlation for individual input dimensions
n_scan = 100
nrows, ncols = 5, 3
figsize = (12, 12 * nrows / ncols)

fig, ax = plt.subplots(nrows=nrows, ncols=ncols, sharey=False, figsize=figsize)
idx = [i for i in range(surrogate.x_dim) if i not in surrogate.fixed_feature_columns]  # remove fixed dimensions

for i in range(nrows * ncols):
    ax_i = ax[i // ncols, i % ncols]
    if i > len(idx) - 1:
        ax_i.axis('off')
    else:
        j = idx[i]

        # map sim values to pv
        parameter_name = surrogate.model_info["model_in_list"][j]
        pv_name = surrogate.pv_info["sim_name_to_pv_name"][parameter_name]
        pv_factor = surrogate.pv_info["sim_to_pv_factor"][parameter_name]
        pv_unit = surrogate.pv_info["pv_unit"][pv_name]
        
        for k in range(3):
            # select reference point
            ref_idx = -(k + 1)
            # ref_idx = k
            if dim == 0:
                ref_x, ref_y = x_x_sorted[ref_idx], y_x_sorted[ref_idx, dim]
            else:
                ref_x, ref_y = x_y_sorted[ref_idx], y_y_sorted[ref_idx, dim]
            
            # calculate 1D scan
            x_scan = scan_ref_x(n_scan, dim=j)
            y_scan = surrogate.model(x_scan).detach()[:, dim]

            # plot data
            ax_i.plot(x_scan[:, j] * pv_factor, y_scan * unit_factor, "C{:d}".format(k))  # label="Ref. Point {:d}".format(k)
            ax_i.axhline(y=0.0, color="gray", linestyle="--")
            ax_i.set_xlabel("{} {}".format(pv_name, pv_unit))

        if i % 3 == 0:
            label = ["sigma_x", "sigma_y"]
            ax_i.set_ylabel("{} {}".format(label[dim], unit))

fig.tight_layout()